<a href="https://colab.research.google.com/github/YuttapongR/128-356-Big-Data_for-test/blob/main/lab_class_4_%E0%B8%A5%E0%B8%B1%E0%B8%9A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ชื่อสมาชิกกลุ่ม ลับ
ยุทธพงศ์ รื่นจิตต์ 6704800002
ปรวัฒน์ ไชยหอม 6704800023
อภิณญา เกตุแก้ว 6704800032

In [1]:
#!/bin/bash
!curl -L -o ./brazilian-ecommerce.zip\
  https://www.kaggle.com/api/v1/datasets/download/olistbr/brazilian-ecommerce

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 42.6M  100 42.6M    0     0   100M      0 --:--:-- --:--:-- --:--:--  100M


In [2]:
!unzip /content/brazilian-ecommerce.zip

Archive:  /content/brazilian-ecommerce.zip
  inflating: olist_customers_dataset.csv  
  inflating: olist_geolocation_dataset.csv  
  inflating: olist_order_items_dataset.csv  
  inflating: olist_order_payments_dataset.csv  
  inflating: olist_order_reviews_dataset.csv  
  inflating: olist_orders_dataset.csv  
  inflating: olist_products_dataset.csv  
  inflating: olist_sellers_dataset.csv  
  inflating: product_category_name_translation.csv  


In [3]:
# ติดตั้ง PySpark (สำหรับ Colab)
!pip -q install pyspark

from pyspark.sql import SparkSession
import os

# สร้าง SparkSession — ประตูเข้าสู่ Spark
# 💡 config("spark.ui.port", "4050") กำหนด port เพื่อเลี่ยง port 4040 ที่อาจชนกัน
spark = SparkSession.builder \
    .appName("BigData-Week4-DeepDive") \
    .config("spark.driver.memory", "2g") \
    .config("spark.ui.port", "4050") \
    .getOrCreate()

# เข้าถึง SparkContext ที่ถูก wrap อยู่ข้างใน
sc = spark.sparkContext

print(f"✅ Spark Version: {spark.version}")
print(f"📱 App Name: {sc.appName}")
print(f"🖥️ Master: {sc.master}")


✅ Spark Version: 4.0.2
📱 App Name: BigData-Week4-DeepDive
🖥️ Master: local[*]


### 🛠️ Task 1: Load & Clean Data

1. อ่านไฟล์ CSV ทั้ง 4 ไฟล์เข้าสู่ Spark DataFrame
2. ตรวจสอบ Schema และจัดการ Type (ถ้าจำเป็น)
3. สร้าง Temp View เพื่อเตรียมพร้อมสำหรับ SQL

In [10]:
prd = spark.read.csv("/content/olist_products_dataset.csv", header=True, inferSchema=True)
od = spark.read.csv("/content/olist_orders_dataset.csv", header=True, inferSchema=True)
cus = spark.read.csv("/content/olist_customers_dataset.csv", header=True, inferSchema=True)

In [15]:
prd.show()

+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|          product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|1e9e8ef04dbcff454...|           perfumaria|                 40|                       287|                 1|             225|               16|               10|              14|
|3aa071139cb16b67c...|                artes|                 44|                       276|                 1|            1000|               30|               18|              20|
|96bd76ec8810374ed...|        esporte_lazer|                 46|                       250|    

In [19]:
prd.createOrReplaceTempView("products")

In [18]:
od.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [17]:
od.createOrReplaceTempView("orders")

In [12]:
cus.show()

+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                    9790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                    1151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|                    8775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|                   13056|            campinas|            SP|
|879864dab9bc30475...|4c93744516667ad3b...|                   89254|      jaragua do sul|            SC|
|fd826e7cf63160e53...|addec96d2e059c80c...|            

In [20]:
cus.createOrReplaceTempView("customers")

### 🔗 Task 2: Join Data

จงสร้าง `df_master` โดยการ Join ตารางดังนี้:
1. `orders` JOIN `items` (ON order_id)
2. JOIN `products` (ON product_id)
3. JOIN `customers` (ON customer_id)

> **Tip:** ตรวจสอบจำนวนแถวหลัง Join ว่าเพิ่มขึ้นหรือลดลงผิดปกติหรือไม่


In [28]:
it = spark.read.csv("/content/olist_order_items_dataset.csv", header=True, inferSchema=True)

df_master = od.join(it, "order_id", "inner") \
              .join(prd, "product_id", "inner") \
              .join(cus, "customer_id", "inner")

print(f"จำนวนแถวของ df_master: {df_master.count():,}")
df_master.show(5)

จำนวนแถวของ df_master: 112,650
+--------------------+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------+--------------------+-------------------+-----+-------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+--------------------+------------------------+---------------+--------------+
|         customer_id|          product_id|            order_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|order_item_id|           seller_id|shipping_limit_date|price|freight_value|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|  c

### 📊 Task 3: Analytics

ตอบคำถามต่อไปนี้ (เลือกใช้ SQL หรือ PySpark ก็ได้):

1. **Top Products:** สินค้าหมวดไหน (`product_category_name`) มียอดขายรวม (price) สูงที่สุด 5 อันดับแรก?
2. **Top Cities:** เมืองไหน (`customer_city`) มีจำนวนคำสั่งซื้อมากที่สุด 5 อันดับแรก?
3. **(Optional) Monthly Sales:** ยอดขายรวมแต่ละเดือนเป็นอย่างไร? (แนวโน้ม)


In [36]:
from pyspark.sql import functions as F

# 1. Top Products: สินค้าหมวดไหนมียอดขายรวม (price) สูงสุด 5 อันดับแรก?
df_master.groupBy("product_category_name") \
         .agg(F.sum("price").alias("total_sales")) \
         .orderBy(F.col("total_sales").desc()) \
         .show(5)

# 2. Top Cities: เมืองไหนมีจำนวนคำสั่งซื้อมากที่สุด 5 อันดับแรก?
# (ใช้ countDistinct order_id เพื่อไม่ให้นับซ้ำ กรณี 1 ออเดอร์มีสินค้าหลายชิ้น)
df_master.groupBy("customer_city") \
         .agg(F.countDistinct("order_id").alias("order_count")) \
         .orderBy(F.col("order_count").desc()) \
         .show(5)

+---------------------+------------------+
|product_category_name|       total_sales|
+---------------------+------------------+
|         beleza_saude| 1258681.340000017|
|   relogios_presentes|1205005.6800000127|
|      cama_mesa_banho|1036988.6800000388|
|        esporte_lazer| 988048.9700000194|
| informatica_acess...| 911954.3200000152|
+---------------------+------------------+
only showing top 5 rows
+--------------+-----------+
| customer_city|order_count|
+--------------+-----------+
|     sao paulo|      15402|
|rio de janeiro|       6834|
|belo horizonte|       2750|
|      brasilia|       2116|
|      curitiba|       1510|
+--------------+-----------+
only showing top 5 rows
